In [1]:
import pandas as pd
import openai
import csv
import numpy as np
from time import sleep
import json
import ast
import re
import random
from itertools import *
import sys
delay = 2

openai.api_key ="API_KEY"

def get_embedding(text, model="text-embedding-ada-002"):
    count = 0 # 예외 발생 횟수를 저장할 변수
    max_count = 5 # 예외가 발생한 최대 횟수
    while True:
        try :
            temp = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
            count=0
            return temp
        except Exception as e:
            count += 1
            print(count)
            if count >= max_count:
                print("예외가 연속으로 %d번 발생했습니다. 프로그램을 종료합니다." % max_count)
                break
            else:
                print("예외가 발생했습니다. (%s)" % e)
def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

def question_function(temp_manual):
    sys = f"""다음의 맥락들을 사용하여 마지막 질문을 생성하세요. 질문을 꾸며내려 하지 마십시오.
질문을 제공할 때는 다음 형식으로 제공되어야 합니다:
메뉴얼: [메뉴얼 내용이 주어짐]
질문: [1. ~~~~~
       2. ~~~~~
       3. ~~~~~ ...]
설명: [질문 생성 이유]

질문을 결정하는 방법:

- 메뉴얼에 있는 내용일수록 더 좋은 질문입니다.
- 충분한 수준의 자세한 내용을 포함하여 질문을 생성합니다.
- 맥락에 따라 메뉴얼 내용을 이해하면서 생성합니다.
- 자신감이 과하지 않도록 주의하세요!
- 질문에 메뉴얼에 없는 내용이 있으면 안됩니다.

예시 #1
메뉴얼:
---------
강의평가
1. 목적

가. 강의에 대한 feedback 통해 강의의 질적 향상 도모
나. 외부교원 재임용시 평가 자료로 활용
다. 교원 업적평가 자료로 활용

2. 시기

정규학기 : 매학기 강의종료달 (6월, 12월) 1일부터 성적확인기간 종료시까지
계절학기 : 학기 마지막 주부터 성적확인기간 종료시까지

3. 대상강좌

해당 학기 전체 개설강좌 (단, 공학인증 강좌는 일반 강의평가뿐 아니라, 공학인증 강의평가도 수행함)

4. 방법

종합정보시스템 - 강좌정보 - 강의평가 메뉴 이용
※ 수강한 전체 과목에 대한 강의평가를 완료하지 않을 경우, 해당학기 성적 조회 불가

5. 강의평가 결과 확인

종합정보시스템 강좌정보 "강의시간표및 계획서"의 강의계획서 내에 전년 동기 강의평가 확인 가능

6. 참고사항

강의평가 응답내용은 익명으로 집계되며 응답자에 대한 비밀이 철저하게 보장됨

문의 : 교무팀/김현수
전화번호02-300-0364
E-mailkhs@kau.ac.kr
---------
질문 : 
---------
1. 강의평가의 목적이 무엇인가요?
2. 강의평가의 시기는 언제인가요?
3. 강의평가의 대상강좌는 무엇인가요?
4. 강의평가의 방법은?
5. 강의평가 결과 확인은 어떻게 하나요?
6. 강의평가의 참고사항은 무엇인가요?
---------
설명 : 주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

예시 #2
메뉴얼:
---------
보험료 청구 방법
기타사항

1. 지원대상 : 학부생, 대학원생

2. 지원내용 : 3백만원 / 1인 / 1사고

- 교내치료비 : 교내 시설물에 의한 신체 상해

※ 보상하지않는 손해

1) 교내치료비 : 교내 시설물에 의한 신체 상해
2) 천재지변으로 생긴 손해에 대한 배상책임
3) 학생들의 개인적인 배상책임
4) 운동선수로 운동을 위한 연습, 경기, 지도 중에 생긴 손해에 대한 배상책임

3. 보험료 청구방법

가. 병원 치료 후 치료비(실손 의료비) 학생본인이 선납
나. 병원치료 완료 후 보험 청구서류 준비 및 서류제출 (피해일로부터 180일 이내)

▶서류 제출처 : 의료지원실(학생회관 2층)
▶청구서류

사고경위서 1부(별첨양식(6) 다운로드)
재학증명서 1부
진단서 또는 의사소견서 1부(서류발급비용 발생)

※ 대체 가능 서류

입원치료시 : 진료비 상세내역서(코드번호 기재) 가능함
통원치료시 : 통원확인서/ 진료확인서/ 진료차트/ 소견서 등 가능함

통장 사본(학생 본인 명의) 1부
진료비 영수증 1부
보험금 청구서(보험회사 지정양식) 1부

다. 보험사 사고조사 및 심사 후 학생계좌로 치료비 송금(약 2~3주 정도 소요)

1. 학생이 원하는 병원에서 진료를 받을 수 있음

2. 치료비 지급은 2년 이내에 보험사에 청구된 부분에 한하여 보상을 받을 수 있음

3. 교내/외 치료비로 지원되는 치료비 항목 : 응급처치비용, 치료, 수술, 영상촬영 등 제반검사, 보철기구를 포함한
치과치료비, 구급차, 입원(건강보험 기준 병실 기준), 병원이 실시한 전문 간호 및 장례비를 말하며 의료보험법 적용대상인 한방치료를 포함함

4. 사고일 당시 재학생이었으나 이후 휴학을 하였으면 재학증명서를 대신 할 ‘재적증명서’를 제출함

5. 문의처 : 학생회관 2층 의료지원실 ☎ 02-300-0024

문의 : 의료지원실/학생회관 2층 224호
전화번호02-300-0024
E-mailoyn@kau.ac.kr
---------
질문 : 
---------
1. 보험료 지원 대상은 누구인가요?
2. 보험료 지원 내용에 대해 알려주세요.
3. 보험료 청구 방법은 무엇인가요?
5. 보험료를 청구할 때 대체 가능 서류에 대해 알려주세요.
---------
설명 : 
---------
주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

예시 #3
메뉴얼:
---------
제 1 학기
구분 : '23.1 January
2023. 1. 1(일) : 신정
2023. 1. 2(월) ~ 6(금) : 복학 집중신청
2023. 1. 2(월) ~ 20(금) : 동계 계절학기
2023. 1. 21(토) ~ 23(월) (1. 24(화) 대체휴일) : 설날

구분 : 2 February
2023. 2. 2(목) ~ 3(금) : 장바구니 수강신청
2023. 2. 9(목) ~ 10(금) : 본 수강신청
2023. 2. 13(월) ~ 17(금) : 휴학 집중신청
2023. 2. 20(월) ~ 24(금) : 등록
2023. 2. 24(금) : 전기 학위수여식(제66회)
2023. 2. 28(화) : 신입생 입학식

구분 : 3 March
2023. 3. 1(수) : 삼일절
2023. 3. 2(목) : 개강
2023. 3. 2(목) ~ 8(수) : 수강신청 변경
2023. 3. 29(수) : 수업일수 4주선

구분 : 4 April

구분 : 5 May
2023. 5. 5(금) : 어린이날
2023. 5. 10(수) : 수업일수 2/3선

구분 : 6 June
2023. 5. 27(일) : 석가탄신일
2023. 6. 6(화) : 현충일
2023. 6. 8(목) ~ 14(수) : 보강기간
2023. 6. 15(목) ~ 22(목) : 기말시험
2023. 6. 16(금) : 개교기념일(제71회)

구분 : 7 July
2023. 7. 3(월) : 하계 계절학기 개강
2023. 7. 3(월) ~ 7(금) : 복학 집중신청
2023. 7. 21(금) : 하계 계절학기 종강

제 2 학기

8월:
- 3일(목) ~ 4일(금): 장바구니 수강신청
- 10일(목) ~ 11일(금): 본 수강신청
- 14일(월) ~ 18일(금): 휴학 집중신청
- 15일(화): 광복절
- 18일(금): 후기 학위수여식(제66회)
- 21일(월) ~ 25일(금): 등록기간
- 28일(월): 개강

9월:
- 28일(월) ~ 1일(금): 수강신청변경
- 24일(일): 수업일수 4주선
- 28일(목) ~ 30일(토): 추석

10월:
- 3일(화): 개천절
- 9일(월): 한글날

11월:
- 5일(일): 수업일수 2/3

12월:
- 4일(월) ~ 8일(금): 보강
- 11일(월) ~ 15일(금): 기말시험
- 25일(월): 성탄절
- 26일(화) ~ 1월 16일(화): 동계 계절학기

24.1월:
- 1일(월): 신정
- 8일(월) ~ 12일(금): 복학 집중신청

24.2월:
- 6일(화) ~ 7일(수): 장바구니 수강신청
- 9일(금) ~ 12일(월): 설날연휴
- 15일(목) ~ 16일(금): 본 수강신청
- 14일(수) ~ 20일(화): 휴학 집중신청
- 23일(금): 전기 학위수여식(제 67회)
- 19일(월) ~ 23일(금): 등록
- 29일(목): 신입생 입학식

문의 : 교무팀/윤미경
전화번호02-300-0011
E-mailmkyoon@kau.ac.kr
---------
질문 : 
---------
1. 제 1학기 1월의 일정에 대해 알려주세요.
2. 제 1학기 2월의 일정에 대해 알려주세요.
3. 제 1학기 3월의 일정에 대해 알려주세요.
4. 제 1학기 4월의 일정에 대해 알려주세요.
5. 제 1학기 5월의 일정에 대해 알려주세요.
6. 제 1학기 6월의 일정에 대해 알려주세요.
7. 제 1학기 7월의 일정에 대해 알려주세요.
8. 2학기 일정에 대해 알려주세요.
9. 24년 일정은 어떻게 되나요?

---------
설명 : 
---------
주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

시작!
메뉴얼:
---------
{temp_manual}
---------
질문: """
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k-0613",
      messages=[
      {"role": "system", "content": f'"{sys}"'},
    ], temperature =  0.001)
    answer = completion['choices'][0]['message']['content']
    return answer 

def answer_function (temp_manual,filter_question):
    sys1 = "assistant는 개발자야"
    user = """당신은 프롬프트 생성 로봇입니다. 당신은 사용자의 목표, 목적, 맥락에 맞는 추가 정보들을 수집해야 합니다.
     프롬프트에는 당신이 제공받은 모든 정보가 모두 포함되어야 합니다.
     당신의 답변은 명확한 형식을 갖추고 있어야 합니다. 
     당신의 답변은 사용자에게 목표, 원하는 출력 형식, 필요한 추가 정보를 묻는 것으로 시작하고 사용자의 목표에 맞게 답변을 해야 합니다.
    """
    ass1 = "원하는 목표와 목적, 원하는 출력 형식 그리고 추가정보가 무엇인가요?"
    user1 = f"""목표: 질문에 대한 답
    목적 : 메뉴얼 내용을 활용하여 답변 구성
    추가정보: 메뉴얼 내용: '{temp_manual}', 질문 내용: '{filter_question}'
    """
    ass2 = f"메뉴얼 내용을 활용하여 답변을 구성하는 것이 목표. 제가 이해한 바로는, 이 주어진 메뉴얼 내용을 적절히 활용하여 답변을 만드는 것인데, 이에 맞게 제가 도와드릴 수 있을까요?"
    user2 = "네 맞아요 답변을 출력해주세요"
    completion = openai.ChatCompletion.create( 
                          model="gpt-3.5-turbo-16k-0613",   
                          messages=[
                          {"role": "system", "content": f'"{sys1}"'},
                          {"role": "user", "content": f'"{user}"'},
                          {"role": "assistant", "content": f'"{ass1}"'},
                          {"role": "user", "content": f'"{user1}"'},
                          {"role": "assistant", "content": f'"{ass2}"'},
                          {"role": "user", "content": f'"{user2}"'},
                          ],temperature = 0
                           ,top_p = 0
                           ,frequency_penalty= 0   # 작을 수록 좀더 길게 답변
                           ,presence_penalty= 1)   # 높을 수록 메뉴얼에 대한 내용이 많이 포함됨
    answer = completion['choices'][0]['message']['content']
    return answer 

Sec_index = []
SecList = []
urlList = []
    
csv_name = "url_data_filter"

data = pd.read_csv(f'{csv_name}.csv', header=None, encoding="UTF-8")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2])
    
###########################################################################################
temp_str = ""
for i in range(len(Sec_index)):
    print(f"------------  {i}  -------------")
    temp_manual = SecList[i]
    err_count = 0
    try :
        question = question_function(temp_manual)
    except Exception as e:
        err_count += 1
        print(err_count)
        if err_count >= 5:
            print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
        else:
            print("예외가 발생했습니다. (%s)" % e)
            question = question_function(temp_manual)     
    result = split_sentences(question)
    filter_result = []
    for j in range(len(result)):
        temp_text = result[j]
        temp_text = temp_text.strip()
        if "----" in temp_text:
            break
        elif temp_text[0].isdigit() and temp_text[1] == '.':
            filter_result.append(temp_text[2:].strip())
        elif temp_text[0].isdigit() and temp_text[1].isdigit() and temp_text[2] == '.':
            filter_result.append(temp_text[3:].strip())
        elif temp_text[-1] == '?':
            filter_result.append(temp_text.strip())
        elif len(temp_text) >=2 and temp_text[-2] == '?':
            filter_result.append(temp_text[:-1].strip())
    for line in filter_result:
        print(f"인덱스 : {i} -> 질문 : {line}")
        err_count = 0
        try :
            answer = answer_function(temp_manual,line)
        except Exception as e:
            err_count += 1
            print(err_count)
            if err_count >= 5:
                print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
            else:
                print("예외가 발생했습니다. (%s)" % e)
                answer = answer_function(temp_manual,line) 
        print(f"인덱스 : {i} -> 답 : {answer}")
        print("----------------------------------")
        f = open(f'url_data_qna.csv','a', newline='' , encoding="utf-8-sig")
        answer_db = csv.writer(f)
        answer_db.writerow([str(Sec_index[i]),temp_manual, line, answer, urlList[i]])
        f.close()                
        

------------  0  -------------
인덱스 : 0 -> 질문 : 복수전공 신청 대상자는 누구인가요?
인덱스 : 0 -> 답 : "복수전공 신청 대상자는 본교 1학년 과정 이상을 수료한 재학생입니다. 추가로 복수전공 신청은 매학기 소정 기간 내에 가능하며, 1월 중에는 1학기 신청이 가능하고, 7월 중에는 2학기 신청이 가능합니다. 자세한 일정은 학사공지의 개별 공지사항을 참고하시면 됩니다. 복수전공 신청 절차는 다음과 같습니다:
1) 복수전공 신청서 작성
2) 소속 학부(과)장 및 부전공 학부(과)장 승인
3) 신청서 교무팀(본관 108호) 제출
4) 복수전공 승인
5) 제출서류로는 부전공 신청서 1부와 성적증명서 1부가 필요합니다.
복수전공 이수에 필요한 학점은 복수전공 학부(과) 전공과목 중에서 42학점 이상으로 요구됩니다. 동일학부 내에서는 36학점 이상의 복수전공 학점이 필요하며, 복수전공 과목 학점 미취득자는 복수전공을 인정하지 않습니다. 복수전공 이수자는 졸업학기말 성적 산출 후 복수전공 학부(과)에서 복수전공 과목 이수를 확인하여 복수전공 이수확인서를 교무팀에 제출하면 됩니다. 주전공 및 융합전공 동일교과목을 중복으로 인정하지 않습니다.
복수전공 신청 관련 양식은 복수전공(신청원, 취소원, 이수확인서)양식을 사용하시면 되며, 복수전공 이수 확인서는 4학년 졸업학기말 성적 산출 후 제출하셔야 합니다.
참고로, 복수전공 변경 및 취소는 매학기 소정 기한 내에 신청 가능하며, 복수전공 신청 전에 이미 이수한 복수전공 교과목도 복수전공 이수로 인정됩니다. 추가 문의 사항이 있으시면 각 학부(과)의 교무팀에 문의하시면 됩니다. 연락처는 02-300-0457이며, 이메일은 kyomu@kau.ac.kr입니다."
----------------------------------
인덱스 : 0 -> 질문 : 복수전공 신청 기간은 언제인가요?
인덱스 : 0 -> 답 : "복수전공 신청 기간은 매학기 소정 기간 내에 있습니다. 1학기는 1월 중, 2학

인덱스 : 2 -> 답 : "전과 신청의 절차는 다음과 같습니다:

1. 신청 대상자:
   - 2학년 전과: 1학년(33학점 이상 이수) 수료 및 2학기 이수자
     [경영학부]: 평점평균 3.0 이상 및 토익 700점 이상
     [항공전자정보공학부]: 평점평균 3.0 이상
     [항공운항학과]: 평점평균 3.5 이상 및 토익 800점 이상

   - 3학년 전과: 2학년(65학점 이상 이수) 수료 및 4학기 이수자
     [경영학부]: 평점평균 3.0 이상 및 토익 700점 이상
     [항공전자정보공학부]: 평점평균 3.0 이상
     [항공운항학과]: 평점평균 3.5 이상 및 토익 800점 이상
     *휴학생은 전과 신청 전 복학신청을 해야 합니다.

2. 신청 기간:
   매년 소정 기간 내에 신청 가능하며, 자세한 일정은 학사공지 '전과 신청 안내'를 참고하시기 바랍니다.

3. 제출 서류 및 제출처:
   - 공통서류: 전과 신청서 1부, 성적증명서 1부
   - [경영학부, 항공교통물류학부]: 토익 성적증명서 1부
   - [항공운항학과]: 전과 지원서 1부, 토익 성적증명서 1부, 항공신체검사증명서 1부
   *제출처는 각 학부(과) 사무실입니다.

4. 신청 절차:
   - 전과 신청서를 각 학부(과) 사무실에 제출합니다.
   - 학부(과)에서 전과 심의를 진행한 후 심사 결과를 개별적으로 통보합니다.
   *면접이 필요한 학과는 해당 학과에서 개별적으로 일정을 통보합니다.

5. 신청서 양식:
   전과 신청원 양식을 사용하시면 됩니다.

추가 문의 사항이 있으시면 각 학부(과)로 문의하시거나 교무팀에 연락해주시기 바랍니다.
관련 문의: 각 학부(과)
관리 담당: 교무팀
전화번호: 02-300-0457
이메일: kyomu@kau.ac.kr

원하시는 출력 형식에 맞게 전과 신청 절차에 대한 정보를 안내해드렸습니다. 추가로 필요한 내용이 있으신가요?"
----------------------------------
인덱스 

인덱스 : 5 -> 질문 : 수강지도 상담의 목적은 무엇인가요?
인덱스 : 5 -> 답 : "수강지도 상담의 목적은 학기 개시 전 다음 학기에 대한 수강계획을 상담하여 학사지도 및 관리를 강화하고, 효율적인 학점 취득을 추구하는 것입니다. 해당 상담을 통해 학생들은 본인이 듣고자 하는 과목을 선택하고, 지도교수는 학생의 수강상담 신청 내역을 검토하여 승인 또는 반려합니다. 수강지도상담은 역량관리시스템 내 수강지도상담 메뉴를 통해서만 진행되며, 이메일이나 전화 등의 기타 방법으로의 상담은 인정되지 않습니다. 수강지도상담을 완료하지 않으면 수강신청이 불가능하니 주의하셔야 합니다. 추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 5 -> 질문 : 수강지도 상담은 어떤 대상을 대상으로 하나요?
인덱스 : 5 -> 답 : "수강지도 상담은 해당학기 수강예정 학부생 전원을 대상으로 합니다. 다만, 학기별 신·편입생은 입학한 첫 학기에는 상담 대상이 아닙니다. 상담은 역량관리시스템 내 수강지도상담 메뉴를 통해 이루어지며, 종합정보시스템 하단 메뉴 &gt; 역량관리시스템 &gt; 통합상담 &gt; 지도교수상담 &gt; 수강지도상담에서 접속할 수 있습니다. 수강지도상담은 학생이 수강하고자 하는 과목의 신청란에 체크하여 신청하며, 본전공 외 교양, 다전공 관련 사항은 전공외 란 및 내용 란에 기입하셔야 합니다. 상담은 지도교수가 학생의 수강상담 신청내역을 검토한 후 승인 또는 반려되며, 승인 시 상담은 완료됩니다. 수강지도상담은 수강신청 전에 완료되어야 하며, 미완료 시 수강신청이 불가능합니다. 추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 5 -> 질문 : 수강지도 상담은 어떤 방법으로 이루어지나요?
인덱스 : 5 -> 답 : "수강지도 상담은 역량관리시스템 내 수강지도상담 메뉴를 통해 이루어집니다. 수강지도상담은 학기별 개설된 전공과목 중 학생이 듣고자

인덱스 : 8 -> 답 : "2021학번의 교양 이수학점은 다음과 같습니다:

- 공통교양: 9 학점
- 학문기초교양: 12~20 학점 (각 학부(과)별로 상이)
- 선택교양: 
   - 철학과 역사, 문학과 예술, 인간과 사회, 실용 및 글로벌 중 최소 3개 영역 선택하여 9 학점 이상 이수
   - 소양, 자기계발 중 자유이수 가능
- 총 이수학점: 39 학점 이상

단, 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목을 포함합니다. 년도별 교양교과과정표를 확인하여 소속 학부(과)별 필수이수 과목을 확인하시기 바랍니다.

추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 8 -> 질문 : 2020학번의 교양 이수학점은 어떻게 되나요?
인덱스 : 8 -> 답 : "2020학번의 교양 이수학점은 다음과 같습니다:

- 공통교양: 7학점
- 학문기초교양: 12~20학점 (각 학부(과)별로 상이)
- 선택교양: 
   - 철학과 역사, 문학과 예술, 인간과 사회, 자연과 과학, 실용 및 글로벌 중 최소 3개 영역 선택하여 9학점 이상 이수
   - 소양, 자기계발 중 자유이수하여 12학점 이상 이수
- 총 이수학점: 39학점 이상

단, 각 학부(과)에 따라 추가적인 이수학점이 필요할 수 있으니, 해당 학부(과)의 교양 이수학점 요구사항을 확인하시기 바랍니다. 추가로 궁금한 사항이 있으면 기초교양교육원에 문의해주시기 바랍니다.

문의:
기초교양교육원
전화번호: 02-300-0014
이메일: mjkwak@kau.ac.kr"

위 답변은 메뉴얼 내용을 참고하여 구성된 것입니다. 추가적인 도움이 필요하시면 언제든지 말씀해주세요.
----------------------------------
인덱스 : 8 -> 질문 : 2018~2019학번의 교양 이수학점은 어떻게 되나요?
인덱스 : 8 -> 답 : "2018~2019 학번의 교양 이수학점은 다음과 같습니다:

- 필수교양: 국어

인덱스 : 9 -> 답 : "사회봉사활동 인정기간은 수강신청 직전학기 기말시험 시작일부터 수강신청 학기 기말시험 전까지입니다. 다만, 헌혈은 기간 제한이 없습니다. 추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 9 -> 질문 : 사회봉사활동 기관 인정 범위는 어떻게 되나요?
인덱스 : 9 -> 답 : "사회봉사활동 기관 인정 범위는 다음과 같습니다:

1. 시·도 및 시·군·구 자원봉사센터, 청소년활동진흥센터, 기타 사회복지관, 지역아동센터, 사회복지법인 사회복지봉사활동인증관리(VMS) 및 1365자원봉사포털에 등록되어 ‘사회복지 봉사활동 실적 인증서(VMS)’ 및 ‘자원봉사활동 확인서(1365)’가 출력되는 기관입니다.

2. 비영리민간단체(봉사활동확인서 제출 시 ‘비영리민간단체등록증’ 사본 첨부)도 인정됩니다.

3. 해외봉사는 인증된 국제 NGO단체에서 인정됩니다. NGO(Non-Governmental Organization 비정부기구)는 지역-국가-국제적으로 조직된 자발적인 비영리 시민단체입니다.

4. 자원봉사센터 및 시민단체와 연계하여 하는 농촌봉사활동도 인정됩니다.

5. 총학생회 주관 자원봉사, 기타 부서 주관 대외행사 및 교내 자원봉사도 인정됩니다.

6. 헌혈활동 역시 인정되며, 1회 기준 4시간까지 인정되고 1학기 최대 2회(8시간)까지만 인정됩니다.

위와 같은 기관과 활동들이 사회봉사활동 기관으로 인정되며, 이를 통해 학점을 인정받을 수 있습니다. 추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 9 -> 질문 : 사회봉사활동 미인정 범위는 무엇인가요?
인덱스 : 9 -> 답 : "사회봉사활동 미인정 범위는 다음과 같습니다:

1. 공공기관, 행정기관에서의 봉사활동
   - 주민센터(구. 동사무소), 시.도.군.구청, 경찰서, 세무서, 소방서, 우체국 등
   - 정부행정기관 : 국세청, 법원, 감사원, 국회 등

2

...

인덱스 : 35 -> 답 : "의료지원실의 운영일시는 월요일부터 금요일까지이며, 운영 시간은 09:00부터 18:00까지입니다. 다만, 점심시간인 12:00부터 13:00까지는 휴무입니다. 의료지원실은 학생회관 2층 224호에 위치해 있으며, 연락처는 02-300-0024(내선 024)입니다. 추가로, 안정실은 이용 가능한 시간은 09:00부터 18:00까지이며, 침대 3개가 제공됩니다. 구급함 대여는 각종 공식행사(동아리 활동, 농활, 체육대회, 교직원 연수 등)시 구급약품의 지원을 통해 진료 기회를 확대하고 있습니다. 대여 방법은 의료지원실을 방문하여 신청하시면 됩니다. 추가 문의 사항이 있으시면 의료지원실/학생회관 224호로 전화(02-300-0024) 또는 이메일(ejp0918@kau.ac.kr)로 문의해주시기 바랍니다."
----------------------------------
인덱스 : 35 -> 질문 : 의료지원실에서 제공하는 시설은 어떤 것들이 있나요?
인덱스 : 35 -> 답 : "의료지원실에서 제공하는 시설은 다음과 같습니다:

1. 응급처치 및 일반의약품 투약: 교내에서 발생한 외상치료와 감기, 소화불량 등 단순증상에 대한 투약(전문의약품 제외)과 응급상황 발생 시 응급처치를 제공합니다.

2. 건강 상담 및 교육: 질병의 조기 발견 및 건강에 대한 올바른 지식을 제공하기 위해 건강 상담과 교육을 진행합니다.

3. 건강증진 프로그램: 금연클리닉(타기관 연계), 일산화탄소 검사, 금연 보조제 제공과 금연 상담 등의 프로그램을 운영합니다.

4. 학교행사 의료지원: 신입생 새로배움터, 축제 등 학교의 공식적인 행사 시 의료지원을 제공합니다.

5. 건강검사: 자동 혈압계, 신장체중계, 혈당계, 시력표를 이용한 검사를 통해 질병의 예방을 돕습니다.

6. 안정실 이용: 학생회관 2층에 위치한 안정실에서 침대 3개를 제공하며, 이용시간은 09:00 ~ 18:00입니다.

7. 구급함 대여: 각종 공식행사(동아리 활동, 농활, 체육대회, 교직

인덱스 : 37 -> 답 : "한국항공대학교의 교육이념은 '개척자적 정신을 배양하고, 지성인으로서의 교양과 첨단의 과학기술로 무장하여, 새로운 시대의 역사적 소임을 다할 수 있는 능력 있는 선도자를 배출' 한다는 건학이념을 바탕으로 합니다. 이러한 건학정신에 따라, 대학은 '인격도야(마음과 몸매를 맑게), 진리탐구(학문과 창의를 넓게), 인화협동(화협과 단결을 굳게)'라는 교육이념을 제정하여, 인재교육을 위한 규범으로 삼고 있습니다. 한국항공대학교는 항공우주과학을 중심으로 국가사회와 인류발전에 필요한 학문분야에 관한 이론과 응용방법을 교수 및 연구하고 아울러 지도자로서의 인격과 실력을 갖춘 유능한 인재를 양성하는 것을 교육목적으로 설정하였습니다. 이를 통해 대학은 국가와 인류사회의 발전에 기여할 인재를 양성하는 고등교육기관으로 역할을 수행하고 있습니다."
----------------------------------
인덱스 : 37 -> 질문 : 한국항공대학교의 교육목적은 무엇인가요?
인덱스 : 37 -> 답 : "한국항공대학교의 교육목적은 '개척자적 정신을 배양하고, 지성인으로서의 교양과 첨단의 과학기술로 무장하여, 새로운 시대의 역사적 소임을 다할 수 있는 능력 있는 선도자를 배출'하는 것입니다. 이는 항공우주과학을 중심으로 국가사회와 인류발전에 필요한 학문분야에 관한 이론과 응용방법을 교수 및 연구하고 아울러 지도자로서의 인격과 실력을 갖춘 유능한 인재를 양성하는 것을 목표로 합니다. 한국항공대학교는 건학이념과 교육이념을 바탕으로 60여년간 항공우주분야의 전문인력을 양성해왔으며, 학생들을 개척자적 정신과 지성인으로 성장시키는데 주력하고 있습니다."
----------------------------------
인덱스 : 37 -> 질문 : 한국항공대학교의 역사와 발전과정에 대해 알려주세요.
인덱스 : 37 -> 답 : "한국항공대학교는 1952년에 개교된 이래로 60여년간 항공우주분야의 전문인력을 양성해왔습니다. 건학이념은 '개척자적 정신을 배양하고, 지

인덱스 : 39 -> 답 : "KAU는 지속 가능한 대학을 달성하기 위해 다양한 노력을 하고 있습니다. 이를 위해 KAU는 Smart & Safe 캠퍼스 구축을 통해 학교 내부의 스마트하고 안전한 환경을 조성하고 있습니다. 또한, 구성원들의 참여와 소통을 기반으로 한 지속 가능한 대학 정립을 추구하고 있습니다. 대학 재정 건전성 확보를 통해 지속 가능한 운영을 지원하고 있으며, 항공우주 분야를 중심으로 한 연구 및 산학 협력을 활성화하여 지속 가능한 열린 대학을 실현하고자 노력하고 있습니다."
----------------------------------
인덱스 : 39 -> 질문 : KAU의 항공우주 특성화 분야에는 어떤 분야들이 포함되나요?
인덱스 : 39 -> 답 : "KAU의 항공우주 특성화 분야에는 다음과 같은 분야들이 포함됩니다:

1. 미래 모빌리티 분야: 드론, UAM (Urban Air Mobility), AI 자율주행 등이 해당 분야에 속합니다.

2. 항공우주 기술 및 운영 분야: 항공우주 기술과 운영에 관련된 분야가 여기에 포함됩니다.

3. 반도체 분야: 반도체 기술과 연구가 항공우주 특성화 분야 중 하나입니다.

4. 교육 혁신: KAU는 창의적이고 융합적인 교육 혁신을 추구하고 있습니다.

이러한 분야들이 KAU의 항공우주 특성화 분야에 포함되어 있습니다. 추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 39 -> 질문 : KAU가 어떻게 항공우주 분야에서 미래 혁신 인재를 양성하고 있는지 알려주세요.
인덱스 : 39 -> 답 : "KAU는 항공우주 분야에서 미래 혁신 인재를 양성하기 위해 다양한 노력을 기울이고 있습니다. 학교는 항공우주 특성화 분야와 미래 모빌리티 분야, 반도체 분야 등에 중점을 두고 교육 혁신을 추진하고 있습니다.

첫째로, 학교는 우수한 교원과 연구인력을 확보하여 항공우주 특성화 연구를 선도하고 있습니다. 이를 통해 학생들은 최신 기술과 동향에 대한 

인덱스 : 43 -> 답 : "엠블럼의 좌우조합형은 엠블럼을 좌우로 나란히 배치한 형태입니다. 이는 한국항공대학교의 심볼인 군청색을 기반으로 송골매가 하늘과 우주로 힘차게 비상하는 모습을 그리고 있습니다. 학교의 국문명과 영문명이 원을 둘레로 각각 새겨져 있으며, 송골매 옆에는 '항대(航大)'라는 한문명이 들어가 있습니다.

상하조합형은 엠블럼을 상하로 나란히 배치한 형태입니다. 역시 군청색을 기반으로 송골매가 하늘과 우주로 힘차게 비상하는 모습을 그리고 있으며, 학교의 국문명과 영문명이 원을 둘레로 각각 새겨져 있습니다. 송골매 옆에는 '항대(航大)'라는 한문명이 들어가 있습니다.

이러한 엠블럼의 좌우조합형과 상하조합형은 다운로드하여 사용할 수 있습니다. 다만, UI를 허가 없이 상업적 목적으로 이용하는 행위는 법률에 저촉되며, UI의 색상, 비율 등 각 요소는 임의로 변형할 수 없습니다.

추가로 궁금한 사항이 있으신가요?"
----------------------------------
인덱스 : 43 -> 질문 : 교색인 군청색은 어떤 의미를 가지고 있나요?
인덱스 : 43 -> 답 : "교색인 군청색은 높고 넓은 하늘을 향한 이상을 뜻하며, 정직과 청결함의 상징으로 상호간의 믿음을 의미합니다. 군청색이 상징하는 맑은 정신은 후세까지 계승하고자 하는 의지를 나타냅니다."
----------------------------------
인덱스 : 43 -> 질문 : 교목인 소나무는 어떤 의미를 가지고 있나요?
인덱스 : 43 -> 답 : "교목인 소나무는 천년이 지나도록 사계절 푸름을 간직하는 나무로, 무수한 고통 속에서도 굳건히 자라나는 기백을 상징합니다. 또한 인간에게 여러모로 도움이 되는 유익한 재료이기도 합니다. 소나무의 이런 굳건함을 본받고자 합니다."
----------------------------------
인덱스 : 43 -> 질문 : 교조인 송골매는 어떤 의미를 가지고 있나요?
인덱스 : 43 -> 답 : "교조인 송골매는 한국항공대의 

인덱스 : 45 -> 답 : "1대 총장 이형재 박사의 임기는 1993.03.01부터 1996.08.07까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 13대 학장 김향욱 박사의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "13대 학장 김향욱 박사의 임기는 1989년 10월 13일부터 1993년 2월 28일까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 12대 학장 김재환 교수의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "12대 학장 김재환 교수의 임기는 1985년 11월 11일부터 1989년 10월 12일까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 10대 학장 한만운 박사의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "10대 학장 한만운 박사의 임기는 1979.03.05부터 1983.02.28까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 9대 학장 송희철 선생의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "9대 학장 송희철 선생의 임기는 1978년 9월 5일부터 1979년 3월 4일까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 5대 학장 김석환 선생의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "5대 학장 김석환 선생의 임기는 1969.02.27부터 1973.02.26까지였습니다."
----------------------------------
인덱스 : 45 -> 질문 : 3대 학장 김노수 선생의 임기는 언제부터 언제까지였나요?
인덱스 : 45 -> 답 : "3대 학장 김노수 선생의 임기는 1960.05.16부터 1962.07.05까지였습니다."
--------

In [ ]:
def question_function(temp_manual):
    sys = f"""다음의 맥락들을 사용하여 마지막 질문을 생성하세요. 질문을 꾸며내려 하지 마십시오.
질문을 제공할 때는 다음 형식으로 제공되어야 합니다:
메뉴얼: [메뉴얼 내용이 주어짐]
질문: [1. ~~~~~
       2. ~~~~~
       3. ~~~~~ ...]
설명: [질문 생성 이유]

질문을 결정하는 방법:

- 메뉴얼에 있는 내용일수록 더 좋은 질문입니다.
- 충분한 수준의 자세한 내용을 포함하여 질문을 생성합니다.
- 맥락에 따라 메뉴얼 내용을 이해하면서 생성합니다.
- 자신감이 과하지 않도록 주의하세요!
- 질문에 메뉴얼에 없는 내용이 있으면 안됩니다.

예시 #1
메뉴얼:
---------
강의평가
1. 목적

가. 강의에 대한 feedback 통해 강의의 질적 향상 도모
나. 외부교원 재임용시 평가 자료로 활용
다. 교원 업적평가 자료로 활용

2. 시기

정규학기 : 매학기 강의종료달 (6월, 12월) 1일부터 성적확인기간 종료시까지
계절학기 : 학기 마지막 주부터 성적확인기간 종료시까지

3. 대상강좌

해당 학기 전체 개설강좌 (단, 공학인증 강좌는 일반 강의평가뿐 아니라, 공학인증 강의평가도 수행함)

4. 방법

종합정보시스템 - 강좌정보 - 강의평가 메뉴 이용
※ 수강한 전체 과목에 대한 강의평가를 완료하지 않을 경우, 해당학기 성적 조회 불가

5. 강의평가 결과 확인

종합정보시스템 강좌정보 "강의시간표및 계획서"의 강의계획서 내에 전년 동기 강의평가 확인 가능

6. 참고사항

강의평가 응답내용은 익명으로 집계되며 응답자에 대한 비밀이 철저하게 보장됨

문의 : 교무팀/김현수
전화번호02-300-0364
E-mailkhs@kau.ac.kr
---------
질문 : 
---------
1. 강의평가의 목적이 무엇인가요?
2. 강의평가의 시기는 언제인가요?
3. 강의평가의 대상강좌는 무엇인가요?
4. 강의평가의 방법은?
5. 강의평가 결과 확인은 어떻게 하나요?
6. 강의평가의 참고사항은 무엇인가요?
---------
설명 : 주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

예시 #2
메뉴얼:
---------
보험료 청구 방법
기타사항

1. 지원대상 : 학부생, 대학원생

2. 지원내용 : 3백만원 / 1인 / 1사고

- 교내치료비 : 교내 시설물에 의한 신체 상해

※ 보상하지않는 손해

1) 교내치료비 : 교내 시설물에 의한 신체 상해
2) 천재지변으로 생긴 손해에 대한 배상책임
3) 학생들의 개인적인 배상책임
4) 운동선수로 운동을 위한 연습, 경기, 지도 중에 생긴 손해에 대한 배상책임

3. 보험료 청구방법

가. 병원 치료 후 치료비(실손 의료비) 학생본인이 선납
나. 병원치료 완료 후 보험 청구서류 준비 및 서류제출 (피해일로부터 180일 이내)

▶서류 제출처 : 의료지원실(학생회관 2층)
▶청구서류

사고경위서 1부(별첨양식(6) 다운로드)
재학증명서 1부
진단서 또는 의사소견서 1부(서류발급비용 발생)

※ 대체 가능 서류

입원치료시 : 진료비 상세내역서(코드번호 기재) 가능함
통원치료시 : 통원확인서/ 진료확인서/ 진료차트/ 소견서 등 가능함

통장 사본(학생 본인 명의) 1부
진료비 영수증 1부
보험금 청구서(보험회사 지정양식) 1부

다. 보험사 사고조사 및 심사 후 학생계좌로 치료비 송금(약 2~3주 정도 소요)

1. 학생이 원하는 병원에서 진료를 받을 수 있음

2. 치료비 지급은 2년 이내에 보험사에 청구된 부분에 한하여 보상을 받을 수 있음

3. 교내/외 치료비로 지원되는 치료비 항목 : 응급처치비용, 치료, 수술, 영상촬영 등 제반검사, 보철기구를 포함한
치과치료비, 구급차, 입원(건강보험 기준 병실 기준), 병원이 실시한 전문 간호 및 장례비를 말하며 의료보험법 적용대상인 한방치료를 포함함

4. 사고일 당시 재학생이었으나 이후 휴학을 하였으면 재학증명서를 대신 할 ‘재적증명서’를 제출함

5. 문의처 : 학생회관 2층 의료지원실 ☎ 02-300-0024

문의 : 의료지원실/학생회관 2층 224호
전화번호02-300-0024
E-mailoyn@kau.ac.kr
---------
질문 : 
---------
1. 보험료 지원 대상은 누구인가요?
2. 보험료 지원 내용에 대해 알려주세요.
3. 보험료 청구 방법은 무엇인가요?
5. 보험료를 청구할 때 대체 가능 서류에 대해 알려주세요.
---------
설명 : 
---------
주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

예시 #3
메뉴얼:
---------
제 1 학기
구분 : '23.1 January
2023. 1. 1(일) : 신정
2023. 1. 2(월) ~ 6(금) : 복학 집중신청
2023. 1. 2(월) ~ 20(금) : 동계 계절학기
2023. 1. 21(토) ~ 23(월) (1. 24(화) 대체휴일) : 설날

구분 : 2 February
2023. 2. 2(목) ~ 3(금) : 장바구니 수강신청
2023. 2. 9(목) ~ 10(금) : 본 수강신청
2023. 2. 13(월) ~ 17(금) : 휴학 집중신청
2023. 2. 20(월) ~ 24(금) : 등록
2023. 2. 24(금) : 전기 학위수여식(제66회)
2023. 2. 28(화) : 신입생 입학식

구분 : 3 March
2023. 3. 1(수) : 삼일절
2023. 3. 2(목) : 개강
2023. 3. 2(목) ~ 8(수) : 수강신청 변경
2023. 3. 29(수) : 수업일수 4주선

구분 : 4 April

구분 : 5 May
2023. 5. 5(금) : 어린이날
2023. 5. 10(수) : 수업일수 2/3선

구분 : 6 June
2023. 5. 27(일) : 석가탄신일
2023. 6. 6(화) : 현충일
2023. 6. 8(목) ~ 14(수) : 보강기간
2023. 6. 15(목) ~ 22(목) : 기말시험
2023. 6. 16(금) : 개교기념일(제71회)

구분 : 7 July
2023. 7. 3(월) : 하계 계절학기 개강
2023. 7. 3(월) ~ 7(금) : 복학 집중신청
2023. 7. 21(금) : 하계 계절학기 종강

제 2 학기

8월:
- 3일(목) ~ 4일(금): 장바구니 수강신청
- 10일(목) ~ 11일(금): 본 수강신청
- 14일(월) ~ 18일(금): 휴학 집중신청
- 15일(화): 광복절
- 18일(금): 후기 학위수여식(제66회)
- 21일(월) ~ 25일(금): 등록기간
- 28일(월): 개강

9월:
- 28일(월) ~ 1일(금): 수강신청변경
- 24일(일): 수업일수 4주선
- 28일(목) ~ 30일(토): 추석

10월:
- 3일(화): 개천절
- 9일(월): 한글날

11월:
- 5일(일): 수업일수 2/3

12월:
- 4일(월) ~ 8일(금): 보강
- 11일(월) ~ 15일(금): 기말시험
- 25일(월): 성탄절
- 26일(화) ~ 1월 16일(화): 동계 계절학기

24.1월:
- 1일(월): 신정
- 8일(월) ~ 12일(금): 복학 집중신청

24.2월:
- 6일(화) ~ 7일(수): 장바구니 수강신청
- 9일(금) ~ 12일(월): 설날연휴
- 15일(목) ~ 16일(금): 본 수강신청
- 14일(수) ~ 20일(화): 휴학 집중신청
- 23일(금): 전기 학위수여식(제 67회)
- 19일(월) ~ 23일(금): 등록
- 29일(목): 신입생 입학식

문의 : 교무팀/윤미경
전화번호02-300-0011
E-mailmkyoon@kau.ac.kr
---------
질문 : 
---------
1. 제 1학기 1월의 일정에 대해 알려주세요.
2. 제 1학기 2월의 일정에 대해 알려주세요.
3. 제 1학기 3월의 일정에 대해 알려주세요.
4. 제 1학기 4월의 일정에 대해 알려주세요.
5. 제 1학기 5월의 일정에 대해 알려주세요.
6. 제 1학기 6월의 일정에 대해 알려주세요.
7. 제 1학기 7월의 일정에 대해 알려주세요.
8. 2학기 일정에 대해 알려주세요.
9. 24년 일정은 어떻게 되나요?

---------
설명 : 
---------
주어진 메뉴얼에서 각각의 세부 내용을 파악하여 빠짐없이 질문을 생성하였습니다. 메뉴얼을 보고 답할 수 있는 질문들입니다.
---------

시작!
메뉴얼:
---------
{temp_manual}
---------
질문: """
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k-0613",
      messages=[
      {"role": "system", "content": f'"{sys}"'},
    ], temperature =  0.001)
    answer = completion['choices'][0]['message']['content']
    return answer 


temp_manual = """2020년대
2010년대
2000년대
1990년대
1980년대
1970년대
1960년대
1950년대

2023.06.16
개교 제 71주년 기념일

2023.05.15
일반대학원 미래항공교통학과 신설

2022.10.18
일반대학원 항공서비스학과 신설

2022.06.16
개교 제 70주년 기념일

2022.01.01
제 9대 총장 허희영 박사 취임

2021.12.31
제 8대 총장 이강웅 박사 퇴임

2021.12.01
인권센터, 현장실습지원센터 설립

2021.06.16
개교 제 69주년 기념일

2021.03.01
AI융합대학 신설- 소프트웨어학과, 스마트드론학과 AI융합대학으로 소속 변경
공과대학 조직개편- 공과대학 내 공학계열 설치- 항공재료공학과 신소재공학과로 명칭변경

2021.02.19
제64회(전기)학위 수여식 거행

2020.09.01
글로벌문화와지역학융합전공, 항공서비스융합전공 신설

2020.08.21
제63회(후기)학위 수여식 거행

2020.06.16
개교 제68주년 기념일

2020.03.01
새내기성공센터 신설

2020.02.21
제63회(전기)학위 수여식 거행

2020.01.01
대학경력개발센터를 대학일자리센터로 통합

2019.08.23
제62회(후기)학위 수여식 거행

2019.06.16
개교 제67주년 기념일

2019.06.01
미래교육혁신원 신설

2019.05.28
제7대 현정택 이사장 취임

2019.03.01
조종융합전공 신설

2019.02.02
제62회(전기)학위 수여식 거행

2018.09.01
공과대학에 공학융합학부 설치 및 항공·경영대학에 항공·경영융합학부 설치
항공경영 융합 전공 신설
항공 MRO 연계전공을 항공정비시스템융합전공으로 개편

2018.08.24
제61회(후기)학위 수여식 거행

2018.02.23
제61회(전기)학위 수여식 거행

2017.08.25
제60회(후기)학위 수여식 거행

2017.06.16
개교 제65주년 기념일

2017.03.13

한국공항공사 계약학과 과정 신설
(일반대학원 항공경영전공 박사과정 10명, 항공경영대학원 항공경영전공 석사과정 40명)

2017. 02. 17
제60회 (전기) 학위수여식 거행

2016. 08. 19
제 59회 (후기) 학위수여식 거행

2016. 06. 16
개교 제 64주년 기념일

2016. 06. 15
개교 제 64주년 기념 행사 실시

2016. 03. 28
생활관 증축

2016. 03. 01
운항 연계전공(30명), 항공MRO 연계전공(30명), 무인기 융합전공(30명) 신설
* MRO : Maintenance, Repair and Overhaul(항공정비산업)

2016. 02. 26
제 59회 (전기) 학위수여식 거행

2015. 09. 03
강의동 준공

2015. 03. 01
공과대학 '항공전자 및 정보통신공학부' 조직 개편('항공전자정보공학부'와 '소프트웨어학과'로 분리)항공·경영대학 '경영학부' 신설('경영학과'와'영어학과'통합)

2015. 02. 27
2015학년도 신입생 입학식 거행

2015. 02. 26
제 58회 (전기) 학위수여식 거행

2014. 10. 23
부속기관 '항공체계시험인증연구센터' 설립

2014. 09. 15
강의동 착공

2014. 08. 22
제 57회 (후기) 학위수여식 거행

2014. 06. 16
개교 제 62주년 기념일

2014. 06. 12
개교 제 62주년 기념 행사 실시

2014. 02. 28
2014학년도 신입생 입학식 거행

2014. 02. 21
제 57회 (전기) 학위수여식 거행

2014. 01. 01
제 7대 총장 이강웅 박사 취임

2013. 12. 27
제 1주기 대학기관평가 인증 취득 (인증기간 : 2014. 1. 1 ∼ 2018.12.31 / 5년)

2013. 08. 23
제 56회 (후기) 학위수여식 거행

2013. 07. 25
학교법인 정석인하학원 출범 (학교법인 정석학원과 학교법인 인하학원 통합 인가)제 6대 조양호 이사장 취임

2013. 07. 16
이강웅 박사 총장 직무대행 취임

2013. 07. 15
제 6대 총장 여준구 박사 퇴임

2013. 06. 16
개교 제 61주년 기념일

2013. 06. 13
개교 제 61주년 기념 행사 실시

2013. 02. 28
2013학년도 신입생 입학식 거행

2013. 02. 15
제 56회 (전기) 학위수여식 거행

2013. 01. 25
부설기관 첨단무인기 연구센터 설립

2012. 12. 31
연구실험동 준공 및 비행교육실습관 2층 증축

2012. 11. 29
CESSNA 항공기 2대 도입

2012. 08. 24
제 55회 (후기) 학위수여식 거행

2012. 06. 16
개교 제 60주년 기념일

2012. 05. 24
개교 제 60주년 기념 행사 실시

2012. 04. 04
모의관제교육장비시스템실(신 장비) 개소식 거행

2012. 04. 01
부설기관 국방광역감시 특화연구센터 설립

2012. 03. 29
부설기관 경기 미니테크노파크 지역특화사업센터 설립

2012. 02. 29
2012학년도 신입생 입학식 거행

2012. 02. 17
제 55회 (전기) 학위수여식 거행

2011. 12. 26
연구실험동 착공식 거행

2011. 06. 16
개교 제 59주년 기념일

2011. 06. 15
개교 제 59주년 기념 행사

2011. 05. 25
울진비행훈련원 격납고 시설 준공

2011. 03. 04
관제소 신축건물 준공

2010. 11. 01
제6대 총장 여준구 박사 취임

2010. 10. 20
인조잔디 운동장 개장식 거행

2010. 10. 11
중앙도서관 1개 층 증축(기존 2층→3층) 완공

2010. 07. 05
비행교육원 울진비행훈련원 개원

2010. 06. 16
개교 제 58주년 기념일

2010. 05. 26
개교 제 58주년 기념 행사

2010. 05. 04
제 5대 지창훈 이사장 취임

2010. 02. 19
제 53회 (전기) 학위수여식 거행

2009. 12. 16
격납고 / 비행교육시설 준공식 거행

2009. 06. 16
개교 제 57주년 기념일

2009. 06. 12
개교 제 57주년 기념 행사

2009. 03. 01
항공산업·경영대학원을 항공·경영대학원으로 명칭 변경항공교통물류학부와 항공우주법학과를 항공·교통·물류·우주법학부(School of Air Transport, Transportation, Logistics, and Air &amp; Space Law)로 통합

2008. 06. 01
부설기관 차세대 방송미디어기술 연구센터 설립

2007. 06. 15
대학본관 건립

2007. 02. 15
대학 영문명칭 변경(Hankuk Aviation University =&gt; Korea Aerospace University)

2006. 11. 01
제5대 총장 여준구 박사 취임

2006. 04. 26
제4대 총장 홍순길 박사 퇴임 / 조성준 박사 총장직무대행 취임

2006. 02. 01
3개 학부 5개 학과 입학정원 890명대학원 석사과정 10개 학과 147명, 박사과정 9개 학과 51명.항공산업정보대학원 7개 학과 47명.경영대학원 3개 학과 30명

2005. 12. 09
학생회관 증축 완공

2005. 10. 01
부설기관 디스플레이 기술교육센터 설립 운영

2005. 09. 01
부설기관 한국어교육원 설립 운영

2005. 03. 01
전자정보통신컴퓨터공학부에서 항공전자 및 정보통신공학부로 학부 명칭 변경

2004. 08. 08
제4대 총장에 홍순길박사 취임

2004. 06. 16
항공우주센터 준공 / 항공우주박물관 건립

2004. 02. 17
한국항공대학교 산학협력단 설립 운영

2003. 09. 01
항공우주법학과 야간에서 주간으로 개편

2003. 08. 01
학부과정 3개 학부, 5개 학과 입학정원 890명 / 대학원 석사과정 9개 학과입학정원 171명.대학원 박사과정 8개 학과 입학정원 27명 / 항공산업정보대학원6개 학과 입학정원 50명.경영대학원 3개 학과 입학정원 과정 75명

2002. 11. 29
항공우주센터 착공식

2002. 04. 30
제4대 재단이사장에 金鍾善 선생 취임

2002. 03. 12
한국항공안전교육원 설립

2001. 08. 23
중앙도서관 및 중소벤쳐육성지원센터 준공

2001. 07. 26
대학원 정원조정에 의거 대학원 석사과정 영어학과 신설, 항공공학과, 항공기계공학과, 기계설계학과, 항공우주공학과를 항공우주 및 기계공학과로 통합하고, 박사과정에 항공교통학과, 경영학과를 신설, 항공공학과, 기계설계학과를 항공우주 및 기계공학과로 통합. 항공기계공학과, 기계설계학과를 항공우주 및 기계공학과로 통합하고 2002학년도 입학정원은 석사과정 인문사회 계열은 17명, 자연과학 계열은 14명, 공학계열은 140명으로, 박사과정 공학계열 21명, 자연과학계열 3명, 인문사회계열 3명으로 조정하고 항공산업대학원을 항공산업정보대학원으로 명칭 변경, 정보보호학과, 항공관광정보학과 신설,경영대학원에 관광경영학과 신설

2001. 06. 01
부설기관 인터넷 정보검색 연구센터 설립

2000. 08. 07
제3대 총장에 홍순길 박사 취임

2000. 02. 25
교직과정 설치(항공우주 및 기계공학부, 항공재료공학과, 전자 · 정보통신 · 컴퓨터공학부) 60명 인가

1999. 03. 22
CESSNA 항공기 4대 도입 및 MOONEY 항공기 1대 대한항공으로부터 수증

1999. 03. 01
대학원 정원 개정에 의거 항공산업대학원 석사과정에 항공산업공학과를 기계설계학과로, 전파공학과를 정보통신학과로 명칭 변경하고, 항공우주법학과를 신설

1998. 11. 05
대학학생정원조정(대학 81413-1284)으로 경영학과 야간40명, 영어학과 야간 30명 신설 총 입학정원 860명

1998. 10. 28
부설기관 중소기업 기술지원센터 설립

1997. 10. 25
대학원 정원개정에 의거 석사과정에 항공우주공학과를 신설하고 공학계열에 16명을 증월하여 152명으로, 박사과정에 항공재료공학과를 신설하고 공학계열에 5명을 증원하여 30명으로 경영대학원(경영학과, 항공경영학과)을 설치하고 입학정원 30명으로 인가, 항공산업대학원의 항공경영학과를 폐지

1997. 10. 19
대학학생정원조정(대제 81413-526)으로 항공우주법학과 야간 30명 신설 총 입학정원 890명

1997. 08. 16
학생회관 준공

1996. 08. 16
과학관, 기계관, 전자관 준공

1996. 08. 16
제2대 총장에 朴五華 박사 취임대학원 정원개정에 의거 97학년도 입학정원을 석사과정 인문사회계열은 5명, 자연과학계열.은 14명, 공학계열은 81명으로 하고 공학계열내 55명올 증원하여 136명으로, 박사과정 자연과학계열은 4명 공학계열은 18명으로, 학연협동과정의 석사과정은 5명.(항공우주연구소 3명, 한국전자통신연구소 2명)으로, 박사과정은 5명(항공우주연구소 1명, 한국전자통신연구소 4명)으로 하고, 학과명칭이 항공경영학과에서 경영학과로 학과명칭 변경, 항공산업대학원 산업전자공학과를 전자공학과로 학과명칭 변경

1996. 03. 01
부속 및 부설기관 재편제.-부속기관 : 중앙도서관, 비행교육원, 출판부, 학보사, 전자계산소, 방송국.-부설기관 : 항공기술교육원, 학생생활연구소, 항공산업기술연구소, 지역사회개발연구소, 전자정보통신개발연구소, 항공산업정책연구소, 기초학문연구소, 항공안전관리연구소, 경영연구소, 사회교육원

1995. 10. 18
대학원 학생정원 조정 및 학과명칭 변경(대행 81413-2407)증원 및 증과 : 석사과정(50명)에 항공교통학과, 박사과정(30명)에 항공운항관리학과, 컴퓨터공학과, 기계설계학과를 신설, 항공산업대학원에 70명 증원하고 일반대학원 석사과정에 전자계산학과를 컴퓨터공학과로 학과명칭 변경

1995. 10. 04
대학학생정원조정(대행 12100-2338)에 의하여 증원 및 학과명칭변경(항공통신정보공학과 50명, 항공전자공학과 40명 증원) 총 입학정원은 670명에서 760명으로 증원. 항공경영학과.에서 경영학과로 학과명칭 변경

1995. 09. 05
사회교육원 신설 학칙변경인가(학무 81412-1227)

1995. 07. 14
제3대 재단이사장에 趙亮鎬 박사 취임

1995. 05. 09
대학원을 대학원 및 항공산업대학원으로, 정비공장, 무선실험국, 항공관제소를 삭제하고 중앙연구소를 신설하는학칙변경(교육부 학무 81412-643)

1994. 10. 21
대학원 정원조정(대행 81413-2071)증원 및 증과 : 석사과정(30명)에 항공경영학과, 항공기계공학과를 신설, 학연산협동과정에 13명(석사3명, 박사10명)증원 및 전문대학원 40명(항공산업대학원 40명)에 항공기계공학과, 산업전자공학과, 항공산업공학과, 전파공학과, 항공재료공학과,항공경영학과 신설

1994. 09. 05
대학 학생정원조정에 의하여 학과명칭 변경(항공관리학과를 항공교통학과로 전자계산학과.를 컴퓨터공학과로)

1993. 09. 04
대학원 학생정원 조정 및 학칙변경(대학 81412-1823호)증원 및 증과 : 석사과정 55명에 항공재료공학과, 기계설계학과, 전자계산학과 및 박사과정(15명)에 항공통신정보공학과를, 학연산협동과정 8명(석사과정 5명, 박사과정 3명) 신설

1993. 09. 03
대학학생 정원조정(대학 85413-1816)(증과 및 증원 ; 항공우주공학과를 신설하여 40명, 항공운항학과 20명, 항공재료공학과 10명, 기계설계학과 20명 증원) (총 입학정원이580명.에서 670명으로 증원)

1993. 03. 01
제1대 총장에 李亨宰 박사 취임

1993. 02. 25
대학을 대학교로, 학장을 총장으로, 항공기술요원양성소를 항공기술교육원으로, 새마을 연구소를 지역사회개발연구소로 변경하고, 기초학문연구소 및 항공안전관리연구소를 신설하는 학칙변경(교육부 학무 81412-350호)

1993. 01. 21
한국항공대학 부설 항공기술교육원을 항공종사자 전문교육기관으로 지정(교통부 운항 91245-42호)

1993. 01. 08
한국항공대학을 한국항공대학교로 학장 및 부학장을 총장 및 부총장으로, 대학원 교학과 및 과장을 교학부 및 부장으로 정관변경(교육부 대학 81422-26호)

1992. 10. 23
새로운 캠퍼스 건설 기공식

1992. 08. 01
제14대 학장에 李亨宰 박사 취임

1992. 02. 29
대학원 학칙변경 인가(명예박사학위 추가) (교육부 학무25210-458호)

1991. 05. 31
부속 및 부설기관 재편재.- 부속기관 : 도서관, 정비공장, 무선실험국, 항공관제소, 출판부, 학보사, 전자계산소, 방송국.- 부설기관 : 항공기술요원양성소, 학생생활연구소, 항공문제연구소, 새마을연구소, 비행교육원, 전자정보통신개발연구소, 항공산업정책연구소

1991. 05. 27
소련 레닌그라드 항공대학과 자매결연, 학술문화 교류 협정 체결(소련측 방문자 : G.A.kryzhanovsky 학장외 부학장 2명)

1989. 10. 13
제 13대 학장에 金恒旭 박사 취임

1989. 06. 01
부설 항공산업정책연구소 설치

1988. 11. 30
대학원 정원개정에 의하여 석사과정 항공운항관리학과를 신설하고 동과정 정원 52명을 64명으로 증원

1987. 11. 09
대학원 박사과정(항공공학과, 항공전자공학과, 정원 18명)설치 인가

1987. 11. 09
대학원 정원개정에 의하여 석사과정에 항공통신정보공학과를 신설하고 동과정 정원 40명을 52명으로 증원

1987. 10. 23
대학학생 정원령 개정에 의하여 증과, 증원(항공운항학과40명, 항공경영학과 70명, 항공. 기계공학과 130명, 항공통신정보공학과 70명, 항공전자공학과 80명, 항공관리학과50명, 항공재료공학과 50명, 기계설계학과 40명, 전자계산학과 50명) 총 9학과에 입학정원 580명.으로 함

1986. 09. 10
부설 전자정보통신개발연구소 설치

1986. 03. 01
부설 비행교육원 설치

1985. 11. 11
제12대 학장에 金載歡교수 취임

1984. 10. 05
항공통신공학과를 항공통신정보공학과로 학과명칭 변경

1984. 10. 01
부속 전자계산소 설치

1983. 03. 01
제11대 학장에 韓萬運 박사 유임

1981. 05. 01
제2대 재단이사장에 趙重建 선생 취임

1980. 10. 02
항공관리학과를 항공경영학과로 학과명칭 변경

1979. 12. 05
대학원 석사과정(항공공학과 20명, 항공전자공학과 20명)설치인가

1979. 03. 05
제10대 학장에 韓萬運박사 취임

1979. 03. 05
학교법인 정석학원 초대 이사장에 趙重勳 박사 취임

1979. 01. 10
국립 한국항공대학교를 학교법인 정석학원에서 인수

1978. 10. 31
입학정원을 100명 증원(항공관리학과 10명, 항공기계공학과 60명, 항공통신공학과 10명, 항공전자공학과 20명)하여 총 입학 정원을 300명으로 함

1978. 09. 05
제9대 학장에 宋훌鐵 선생 취임

1977. 08. 25
제8대 학장에 宋孝聊 선생 유임

1976. 12. 31
대학학생 정원령 개정(대통령령 제8393호)에 의거 항공운항학과, 항공관리학과 입학정원을 각 10명씩 증원하여 총 입학 정원을 200명으로 함

1975. 08. 30
운항학관 준공

1974. 12. 28
기계공학관 준공

1974. 06. 01
부속 항공관제소 설치

1973. 08. 25
제7대 학장에 宋孝聊 선생 취임

1973. 02. 27
제6대 학장에 金右桓 선생 유임

1972. 12. 30
도서관 준공

1972. 12. 22
대학학생 정원령 개정에 의거 운항관리학과를 분리 항공운항학과 30명, 항공관리학과 30명으로 하여 총 입학정원을 180명으로 함

1971. 12. 18
대학학생 정원령 개정에 의거 정비공학과를 항공기계공학과로 학과명칭을 변경하고, 항공기계공학과, 항공통신정보공학과, 항공전자공학과의 입학정원을 각각 10명씩증원, 총 입학정원을 150명으로 함

1971. 12. 10
공군 R.O.T.C. 설치

1970. 12. 18
교양학관 준공

1970. 12. 22
각 학과 입학정원올 10명씩 증원, 총 입학정원 120명으로 함

1969. 10. 25
부속 학생지도연구소 설치

1969. 02. 27
제5대 학장에 金石桓 선생 취임

1968. 12. 31
국립학교 설치령(대통령령 제3696호) 개정에 의거 교통부 소관 국립항공학교를 문교부로 이관하고 한국항공대학으로 개편, 조종과, 기체기관과, 통신관제과, 응용전자과를 각각 운항관리학과, 정비공학과, 항공통신공학과, 항공전자공학과로 학과명칭 변경, 각 학과 입학정원을 20명으로함. 부속기관으로 도서관, 정비공장, 무선실험국, 항공문제연구소와 부설기관으로 항공기술요원양성소를 설치

1968. 09. 02
통신전자공학관 준공

1965. 01. 19
국립학교 설치령 일부 개정(대통령령 제2039)에 의거 입학자격을 고등학교 졸업자 또는 동등 이상의 자로 하고 수업년한 4년제로 개편

1964. 03. 31
문교부 고시 제191호에 의거 이공계 4년제 대학 졸업자와 동등의 학력을 인정받음

1963. 11. 22
구 교통부 시설국 청사에서 신축교사(현 경기도 고양시 덕양구 화전동 200-1)를 준공, 교사이전

1963. 10. 23
국립학교 설치령(각령 1605)에 의거 국립항공학교로 개편, 입학자격을 일반 이공계 대학 2년 수료자또는 초급대학 졸업자로 하고 대학 3, 4학년 과정의 2년제로 개편, 특과를부설

1962. 08. 01
서울특별시 용산구 한강로 40번지(구 교통부 시설국 청사)로 교사 이전

1962. 07. 06
제4대 학장에 宋孝聊선생 취임(4대)

1960. 05. 16
제3대 학장에 金魯洙선생 취임(3대)

1959. 12. 31
조종관제과를 조종과로, 통신전자과를 분리하여 통신관제과, 응용전자과로 과명칭 변경

1958. 01. 14
조종과, 기관과, 통신과를 각각 조종관제과, 기체기관과, 통신전자과 과명칭 변경

1955. 06. 18
훈련비행장을 준공하고, 본 대학 훈련비행기(제1기 기판과 학생 조립)에 의한 처녀 비행을 함

1954. 08. 03
항공기 격납고 준공(고양군 신도면 화전리 200-1)

1954. 01. 25
제2대 학장에 宋孝柳선생 취임

1953. 11. 14
수업연한 4년제 국립항공대학으로 개편

1953. 10. 15
서울특별시 용산구 한강로 3가 65번지에 기숙사 준공과 함께 이전

1952. 06. 16
수업년한 3년제로 개편하여 국립항공학교로 독립, 부산시 범일동 가교사로 이전. 초대 교장.에 金允基선생 취임

1952. 06. 16
6.25 동란 중 민항공분야의 개척을 위한 교육기관이 필요함에 따라 교통부 산하 교통고등학교 2년제 특설 항공과(조종과, 기관과, 통신과)로 발족(부산시 초량동:교장 權容韓 교장겸임)

문의 : 총무팀/강문구
전화번호02-300-0033
E-mailmoon9@kau.ac.kr"""

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

answer = question_function(temp_manual)   
result = split_sentences(answer)
filter_result = []
for j in range(len(result)):
    temp_text = result[j]
    temp_text = temp_text.strip()
    if "----" in temp_text:
        break
    elif temp_text[0].isdigit() and temp_text[1] == '.':
        filter_result.append(temp_text[2:].strip())
    elif temp_text[0].isdigit() and temp_text[1].isdigit() and temp_text[2] == '.':
        filter_result.append(temp_text[3:].strip())
    elif temp_text[-1] == '?':
        filter_result.append(temp_text.strip())
    elif len(temp_text) >=2 and temp_text[-2] == '?':
        filter_result.append(temp_text[:-1].strip())
for line in filter_result:
    print(line)

In [ ]:
print(answer)